In [ ]:
import sys
import os
import random
import mxnet as mx

## DataIter

mx.io.DataIter 用于提供输入数据的接口

In [ ]:
mnist = mx.test_utils.get_mnist()
batch_size = 1
print("data shape {} and data type {}".format(mnist["train_data"].shape, type(mnist["train_data"])))
train_iter = mx.io.NDArrayIter(mnist["train_data"], mnist["train_label"], batch_size, shuffle=True)
print("provide_data:", train_iter.provide_data)
print("provide_label:", train_iter.provide_label)
count = 0
while True:
    try:
        train_iter.next()
        count += 1
        if count % 10000 == 0:
            print("Current batch number:", count)
    except StopIteration:
        print("Read to End")
        break


多轮遍历方法一

In [ ]:
epoch = 0
N = 10
train_iter.reset()
while True:
    try:
        train_iter.next()
    except StopIteration:
        train_iter.reset()
        epoch += 1
        print("Epoch num ", epoch)
        if epoch >= 10:break

多轮遍历方法二

In [ ]:
for i in range(N):
    train_iter.reset()
    for batch in train_iter:
        pass
    print("Epoch num ", i+1)

## 图片输入格式
将图片文件转化成 id label filename 格式

In [ ]:
def make_dogvcat_data(data_loc, set_name):

    # FORMAT ::= int_image_index \t label_index \t path_to_image \n

    image_files = os.listdir(data_loc)
    random.seed(100)
    random.shuffle(image_files)

    n_image = len(image_files)
    n_train = int(n_image * 0.8)
    n_test = n_image - n_train

    # train
    fout = open(os.path.join(data_loc+'/../', set_name+'_train.lst'), 'w')

    DOG = 0
    CAT = 1
    for i in range(n_train):
        filename = image_files[i]
        label = DOG if 'dog' in filename else CAT
        fout.write('%d\t%d\t%s\n'%(i, label, filename))
    fout.close()

    # test
    fout = open(os.path.join(data_loc+'/../', set_name+'_test.lst'), 'w')
    for i in range(n_test):
        filename = image_files[n_train+i]
        label = DOG if 'dog' in filename else CAT
        fout.write('%d\t%d\t%s\n'%(i, label, filename))
    fout.close()

make_dogvcat_data("data/cats_vs_dogs/train_data", "cats_vs_dogs")

In [ ]:
images = []
for line in open("data/cats_vs_dogs/cats_vs_dogs_train.lst"):
    id,label,path = line.rstrip().split()
    images.append("data/cats_vs_dogs/train_data/" + path)
print images
    

## 调用im2vec.py 输出 vec格式
> -resize xxx   resize图片

python im2vec.py data/cats_vs_dogs/cats_vs_dogs_train  data/cats_vs_dogs/train_data

执行后，会生成 data/cats_vs_dogs/cats_vs_dogs_train.vec 文件


In [ ]:
%matplotlib inline
import mxnet as mx
import matplotlib.pyplot as plt
import cv2

data_shape = (3,228,228)
train = mx.io.ImageRecordIter(
        path_imgrec = "data/cats_vs_dogs/cats_vs_dogs_train.rec",
        data_shape  = data_shape,
        data_name   = 'data',
        label_name  = 'softmax_label',
        batch_size  = 4,
        resize = 250,
        rand_crop   = True,
        rand_mirror = True,
        shuffle = False,
        num_parts   = 1,
        part_index  = 0
)
# 打印iter的data和label信息
print train.provide_data
print train.provide_label
batch = train.next()


In [ ]:
_, figs = plt.subplots(2, 2)

for i in range(2):
    for j in range(2):
        img = cv2.imread(images[2*i+j],1)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        fig = figs[i][j]
        print img.shape
        fig.imshow(img)
plt.show()


In [ ]:
_, figs = plt.subplots(2, 2)
for i in range(2):
    for j in range(2):
        img, labels = batch.data[0][2*i+j], batch.label[0][2*i+j]
        img = img.transpose((1, 2, 0)) 
        img = img.clip(0,255).asnumpy()/255.0
        fig = figs[i][j]
        fig.imshow(img)
plt.show()